In [1]:
# problem 3
n = 3
a = 7
x = 1

# list all possible y we can get with measurement
ys = []
for y in range(2 ** n):
    t = 0
    for i in range(n):
        t ^= (((y & a) >> i) & 1)
    if t == 0:
        ys.append(y)
print(ys)

# compute number of independent vectors
def nindependent(vs):
    b = [0] * n # basis
    answer = 0
    for v in vs:
        x = v
        for i in range(n):
            if ((x >> i) & 1) == 1:
                if b[i] == 0:
                    b[i] = x
                    answer += 1
                    break
                else:
                    x ^= b[i]
    return answer

# choosing n + x = 4 random vectors
total = len(ys) ** 4
good = 0
for y1 in ys:
    for y2 in ys:
        for y3 in ys:
            for y4 in ys:
                if nindependent([y1, y2, y3, y4]) == 2:
                    good += 1
print(good / total)

# computing by formula
q = 1
for i in range(2, n + 1):
    q *= (1 - 1 / (2 ** (i + x)))
print(q)

[0, 3, 5, 6]
0.8203125
0.8203125
